In [1]:
import pandas as pd

In [2]:
cc_info = pd.read_csv('cc_info.csv')
tran = pd.read_csv('transactions.csv')
cc_info.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [3]:
tran.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


#### EDA

In [4]:
cc_info.isnull().sum()

credit_card          0
city                 0
state                0
zipcode              0
credit_card_limit    0
dtype: int64

In [5]:
tran.isnull().sum()

credit_card                  0
date                         0
transaction_dollar_amount    0
Long                         0
Lat                          0
dtype: int64

In [6]:
len(cc_info)

984

In [7]:
len(tran)

294588

#### Data Prepration

In [8]:
tran_detail = pd.merge(left = cc_info, right = tran, how = 'left',left_on = 'credit_card',right_on = 'credit_card')
tran_detail['date'] = pd.to_datetime(tran_detail['date'])

In [9]:
#date extract
tran_detail['month'] = tran_detail['date'].apply(lambda x:x.month)
tran_detail['day'] = tran_detail['date'].apply(lambda x:x.day)
tran_detail['DoW'] = tran_detail['date'].apply(lambda x:x.dayofweek)
tran_detail.head()

,credit_card,city,state,zipcode,credit_card_limit,date,transaction_dollar_amount,Long,Lat,month,day,DoW
0,1280981422329509,Dallas,PA,18612,6000,2015-08-05 00:59:19,11.94,-75.964527,41.353578,8,5,2
1,1280981422329509,Dallas,PA,18612,6000,2015-10-29 18:23:04,5.76,-76.019703,41.311467,10,29,3
2,1280981422329509,Dallas,PA,18612,6000,2015-10-25 17:50:48,25.84,-76.043031,41.291053,10,25,6
3,1280981422329509,Dallas,PA,18612,6000,2015-09-05 17:39:43,68.89,-75.944299,41.327282,9,5,5
4,1280981422329509,Dallas,PA,18612,6000,2015-09-04 14:12:59,27.01,-75.997259,41.352099,9,4,4


#### List of people who doesn't go over the limit at all time

In [10]:
tran_detail_Q1 = tran_detail.groupby(['credit_card','month']).apply(lambda x: pd.Series([x['transaction_dollar_amount'].sum(),x['credit_card_limit'].values[0]],index = ['total_tran','credit_limit'])).reset_index()
tran_detail_Q1.head()

,credit_card,month,total_tran,credit_limit
0,1003715054175576,7,162.56,20000.0
1,1003715054175576,8,8441.51,20000.0
2,1003715054175576,9,11281.56,20000.0
3,1003715054175576,10,8954.21,20000.0
4,1013870087888817,7,281.43,15000.0


In [11]:
tran_detail_Q1['exceed'] = tran_detail_Q1['total_tran'] > tran_detail_Q1['credit_limit']
Q1 = tran_detail_Q1.groupby('credit_card')['exceed'].sum().reset_index()
lists = Q1[Q1['exceed'] == 0]['credit_card']

In [12]:
lists

0      1003715054175576
1      1013870087888817
2      1023820165155391
3      1073931538936472
4      1077622576192810
             ...       
978    9946917923319410
979    9958678964376192
980    9961694231875562
981    9981251982982618
982    9986135779184360
Name: credit_card, Length: 862, dtype: int64

On the other hand, she wants you to implement an algorithm that as soon as a user goes above her monthly limit, it triggers an alert so that the user can be notified about that. We assume here that at the beginning of the new month, user total money spent gets reset to zero (i.e. she pays the card fully at the end of each month). Build a function that for each day, returns a list of users who went above their credit card monthly limit on that day.

In [13]:
def alert(data, month, day):
    """ function to return the list of credit cards that exceed the limit """
    data = data[(data['month'] == month) & (data['day'] <= day)]
    limit = data.groupby('credit_card')['credit_card_limit'].min().reset_index()
    transaction = data.groupby('credit_card')['transaction_dollar_amount'].sum().reset_index()
    
    merged = pd.merge(left=transaction, right=limit, on='credit_card', how='left')
    result = merged[merged['transaction_dollar_amount'] > merged['credit_card_limit']]
    
    return result

In [14]:
# example
alert(tran_detail, month=8, day=20)

,credit_card,transaction_dollar_amount,credit_card_limit
194,2850146878241916,4012.81,4000
319,3936887050542904,5352.07,4000
341,4118286032166087,3416.52,3000
383,4462290711836916,3119.53,2000
525,5723635641134781,2161.30,2000
698,7299183791723634,7074.49,7000
772,7850942767136368,5704.00,4000
834,8522875529951473,5860.89,5000
874,8972201384562696,5266.08,5000
